In [2]:
import pandas as pd
import numpy as np
import os
import re
import subprocess
from glob import glob

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from rdkit import Chem
from rdkit.Chem import Descriptors, Descriptors3D, ChemicalFeatures, GraphDescriptors, Lipinski, rdchem
from rdkit.Chem.rdchem import Mol, Atom, Bond
from rdkit.Chem.rdmolfiles import SDMolSupplier, SDWriter
from rdkit.rdBase import BlockLogs
import datamol as dm
import multiprocessing
import networkx as nx

%matplotlib inline

temp_path = './temp'
data_path = './data'
graph_data_path = './graph_data'

if not os.path.exists(temp_path):
    os.mkdir(temp_path)

In [3]:
xyz_filepath_list = list(glob(f'{data_path}/*.xyz'))

xyz_filepath_list.sort()
print('total xyz filepath # ', len(xyz_filepath_list))
xyz_filepath_list[0]

total xyz filepath #  133885


'./data/dsgdb9nsd_000001.xyz'

In [4]:
def check_missing_files() -> None:
    """
    Checks for missing .xyz QM9 files

    Return
        True | False: True, if all files are accounted for
    """
    for file in tqdm(xyz_filepath_list):
        if not os.path.exists(file):
            print('Retrieving QM9..')
            !wget https://figshare.com/ndownloader/files/3195389 -O ./data/data.bz2
            !tar -xfu ./data/data.bz2
            !rm ./data/*.bz2

    if not os.path.exists(f'{data_path}/edrug3d.sdf'):
        print('Retrieving edrug3d..')
        !wget https://chemoinfo.ipmc.cnrs.fr/TMP/tmp.33880/e-Drug3D_2056.sdf -O ./data/edrug3d.sdf

    return None


check_missing_files()

  0%|          | 0/133885 [00:00<?, ?it/s]

In [5]:
def in_ipython():
    try:
        return __IPYTHON__
    except NameError:
        return False

In [6]:
manager = multiprocessing.Manager()
molecules = manager.list()


def xyz2mol(m):
    blockedLogs = BlockLogs()

    with open(m) as file:
        lines = file.readlines()

        smiles = lines[len(lines) - 2].split()[0]
        smiles = dm.standardize_smiles(smiles)

        mol = dm.to_mol(smiles, add_hs=True, sanitize=True, ordered=True)
        mol = dm.fix_mol(mol)
        dm.align.compute_2d_coords(mol)
        # dm.conformers.generate(mol)

        molecules.append(mol)


def e3d2mol():
    blockedLogs = BlockLogs()

    mols = dm.read_sdf(f'{data_path}/edrug3d.sdf', remove_hs=False)
    molecules.extend(mols)


# Not working yet
# e3d2mol()

dm.parallelized(xyz2mol,
                xyz_filepath_list,
                n_jobs=-1,
                progress=True,
                arg_type='arg',
                total=len(xyz_filepath_list))

print('Done')

  0%|          | 0/133885 [00:00<?, ?it/s]

Done


In [7]:
len(molecules)

133885

In [46]:
def preprocess():
    graphs_df = None
    nodes_df = None
    bonds_df = None

    line_split_re = re.compile(r'\s+')

    index = 0

    for mol in tqdm(molecules):
        mol: rdchem.Mol

        if mol is None:
            continue

        mol = Chem.AddHs(mol)

        # Mol.Compute2DCoords(mol)

        mol_nodes_df = None
        mol_bonds_df = None

        with open(f'{temp_path}/{index}_mol.pdb', 'w') as file:
            Chem.MolToPDBFile(mol, f'{temp_path}/{index}_mol.pdb')

        # Run antechamber and divert output to a file (temporary)
        subprocess.getoutput(
            f'cd {temp_path} && antechamber -i {index}_mol.pdb -fi pdb -o {index}_mol.ac -fo ac -at gaff2 -pf y'
        )

        # Compute molecule-level info
        graphs_args = {
            'graph_id': index
        }

        mol_graph_df = pd.DataFrame(graphs_args, index=[0])

        with open(f'{temp_path}/{index}_mol.ac') as file:
            lines = [a for a in file.readlines() if 'ATOM' in a]

            for atom in Mol.GetAtoms(mol):
                atom: rdchem.Atom

                atom_idx = atom.GetIdx()

                line = lines[atom_idx]

                a_split = line_split_re.split(line.strip())

                if len(a_split) > 9:
                    atom_type = a_split[9]
                else:
                    atom_type = a_split[8]

                atom_args = {
                    'graph_id': index,
                    'node_id': atom_idx,
                    'type': atom_type,
                    'label': atom.GetSymbol(),
                    'depDeg': atom.GetDegree(),
                    'forC': atom.GetFormalCharge(),
                    'isA': float(atom.GetIsAromatic()),
                    'radEl': atom.GetNumRadicalElectrons(),
                    'totDeg': atom.GetTotalDegree(),
                    'totH': atom.GetTotalNumHs(),
                    'totV': atom.GetTotalValence(),
                    'isR': float(atom.IsInRing()),
                }

                # Append molecule info to every atom
                atom_df = pd.DataFrame(atom_args, index=[0])

                if mol_nodes_df is None:
                    mol_nodes_df = pd.DataFrame(columns=atom_df.columns)

                mol_nodes_df = pd.concat([mol_nodes_df, atom_df], ignore_index=True)

        for bond in mol.GetBonds():
            bond: rdchem.Bond

            bond_args = {
                'graph_id': index,
                'src_id': bond.GetBeginAtomIdx(),
                'dst_id': bond.GetEndAtomIdx(),
                'label': bond.GetBondType(),
                'bond_type': bond.GetBondTypeAsDouble()
            }

            atom_bonds_df = pd.DataFrame(bond_args, index=[0])

            if mol_bonds_df is None:
                mol_bonds_df = pd.DataFrame(columns=atom_bonds_df.columns)

            mol_bonds_df = pd.concat([mol_bonds_df, atom_bonds_df], ignore_index=True)

        # print(mol_bonds_df)
        # print(mol_nodes_df)

        if graphs_df is None:
            graphs_df = pd.DataFrame(columns=mol_graph_df.columns)

        if nodes_df is None:
            nodes_df = pd.DataFrame(columns=mol_nodes_df.columns)

        if bonds_df is None:
            bonds_df = pd.DataFrame(columns=mol_bonds_df.columns)

        graphs_df = pd.concat([graphs_df, mol_graph_df], ignore_index=True)
        nodes_df = pd.concat([nodes_df, mol_nodes_df], ignore_index=True)
        bonds_df = pd.concat([bonds_df, mol_bonds_df], ignore_index=True)

        # Clean up remaining files
        try:
            os.remove(f'{temp_path}/{index}_mol.ac')
            os.remove(f'{temp_path}/{index}_mol.pdb')
        except IOError as e:
            print(e)
            print('Something went wrong.')

        index += 1

    graphs_df.to_csv(f'{graph_data_path}/graphs.csv')
    nodes_df.to_csv(f'{graph_data_path}/nodes.csv')
    bonds_df.to_csv(f'{graph_data_path}/bonds.csv')

In [47]:
def main():
    preprocess()

In [ ]:
if __name__ == "__main__":
    if not in_ipython():
        root_dir = os.path.dirname(os.path.realpath(__file__))

        main()
    else:
        main()

  0%|          | 0/133885 [00:00<?, ?it/s]